<a href="https://colab.research.google.com/github/PaulAndre1/Paul-Paul-fcc-sms-text-classification/blob/main/Paul_fcc_sms_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.8/47.8 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 633.7/633.7 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 40.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 44.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 347.7/347.7 kB 15.9 MB/s eta 0:00:00
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.2.0
    Uninstalling ml-dtypes-0.2.0:
      Successfully uninstalled ml-dtypes-0.2.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.15.0 requires ml-dtypes~=0.2.0, but you have ml-dtypes 0.4.0 which is incompatible.
2.18.0-dev20240727


In [2]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

--2024-07-27 23:51:25--  https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.2.33, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358233 (350K) [text/tab-separated-values]
Saving to: ‘train-data.tsv’

train-data.tsv      100%[===================>] 349.84K  --.-KB/s    in 0.03s   

2024-07-27 23:51:25 (12.3 MB/s) - ‘train-data.tsv’ saved [358233/358233]

--2024-07-27 23:51:25--  https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.2.33, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118774 (116K) [text/tab-separated-values]
Saving to: ‘valid-data.tsv’

valid-data.tsv      100%[==========

In [3]:
train = pd.read_csv(train_file_path, delimiter = '\t', names = ['class', 'text'])
validation = pd.read_csv(test_file_path, delimiter = '\t', names = ['class', 'text'])

In [63]:
train

,class,text
0,ham,ahhhh...just woken up!had a bad dream about u ...
1,ham,you can never do nothing
2,ham,"now u sound like manky scouse boy steve,like! ..."
3,ham,mum say we wan to go then go... then she can s...
4,ham,never y lei... i v lazy... got wat? dat day ü ...
...,...,...
4174,ham,just woke up. yeesh its late. but i didn't fal...
4175,ham,what do u reckon as need 2 arrange transport i...
4176,spam,free entry into our £250 weekly competition ju...
4177,spam,-pls stop bootydelious (32/f) is inviting you ...


In [64]:
map_classes = {
    'ham': 0,
    'spam': 1
}

In [65]:
df_train = train.copy()
df_validation = validation.copy()

df_train['class'] = df_train['class'].map(map_classes).astype(float)
df_validation['class'] = df_validation['class'].map(map_classes).astype(float)

In [66]:
max_features = 10000
sequence_length = 100

text_vectorizer = tf.keras.layers.TextVectorization(
    max_tokens=max_features,
    output_sequence_length=sequence_length
)

In [67]:
df_train = df_train.sample(frac=1).reset_index(drop=True)

In [68]:
text_vectorizer.adapt(df_train['text'])

In [69]:
def vectorize_text(texts):
    return text_vectorizer(texts)


vectorized_texts_validation = vectorize_text(df_validation['text'])
vectorized_texts = vectorize_text(df_train['text'])
vectorized_texts[1]

<tf.Tensor: shape=(100,), dtype=int64, numpy=
array([   3,   47,   18,  109,   15,   14, 2186,    9,   12, 3390,   65,
          9,  131,   50,    4,  433, 1390, 1404,   94,    4,   35,    3,
        182,  165,    9,   12,  319,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0])>

In [70]:
df_train['text'][1]

"i dont have any of your file in my bag..i was in work when you called me.i 'll tell you if i find anything in my room."

In [71]:
vocab_size = len(text_vectorizer.get_vocabulary())

In [72]:
X = vectorized_texts
y = df_train['class'].values

In [73]:
X_val = vectorized_texts_validation
y_val = df_validation['class'].values

In [74]:
X[0]

<tf.Tensor: shape=(100,), dtype=int64, numpy=
array([  94,   14,  226,   54,    4,  444,    2,   30,   19, 1026,   67,
       2093,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0])>

In [75]:
y

array([0., 0., 0., ..., 0., 1., 0.])

In [76]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, 100),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss="binary_crossentropy", metrics=['acc'])

history = model.fit(X, y, epochs=5, validation_data = (X_val, y_val))

Epoch 1/5
131/131 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - acc: 0.8754 - loss: 0.3487 - val_acc: 0.9670 - val_loss: 0.1099
Epoch 2/5
131/131 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - acc: 0.9820 - loss: 0.0711 - val_acc: 0.9813 - val_loss: 0.0674
Epoch 3/5
131/131 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - acc: 0.9940 - loss: 0.0293 - val_acc: 0.9856 - val_loss: 0.0512
Epoch 4/5
131/131 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - acc: 0.9965 - loss: 0.0185 - val_acc: 0.9842 - val_loss: 0.0528
Epoch 5/5
131/131 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - acc: 0.9984 - loss: 0.0080 - val_acc: 0.9871 - val_loss: 0.0429


In [77]:
model.predict(X)

131/131 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


array([[9.1815758e-03],
       [4.8459816e-04],
       [1.2015028e-02],
       ...,
       [7.5074314e-04],
       [9.9946833e-01],
       [1.8508850e-03]], dtype=float32)

In [78]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  processed_text = vectorize_text([pred_text])
  prediction = model.predict([processed_text])
  print(prediction)
  map_respones = {
      0: 'ham',
      1: 'spam'
  }
  predicted_class = 1 if prediction[0][0] > 0.5 else 0
  return [prediction[0][0], map_respones[predicted_class]]

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
[[0.00244359]]
[0.002443589, 'ham']


In [79]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    print(prediction, passed)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
[[0.00244359]]
[0.002443589, 'ham'] True
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
[[0.58953196]]
[0.58953196, 'spam'] True
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
[[0.00035826]]
[0.0003582643, 'ham'] True
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
[[0.9880329]]
[0.9880329, 'spam'] True
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
[[0.99801743]]
[0.99801743, 'spam'] True
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
[[0.00106286]]
[0.0010628636, 'ham'] True
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
[[0.00327769]]
[0.0032776946, 'ham'] True
You passed the challenge. Great job!
